In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import matplotlib.ticker as ticker
import warnings

In [2]:
df = pd.read_csv('/content/ExpVinho.csv',sep=';')
df.head(10)

,Id,País,1970,1970.1,1971,1971.1,1972,1972.1,1973,1973.1,...,2019,2019.1,2020,2020.1,2021,2021.1,2022,2022.1,2023,2023.1
0,1,Afeganistão,0,0,0,0,0,0,0,0,...,0,0,0,0,11,46,0,0,0,0
1,2,África do Sul,0,0,0,0,0,0,0,0,...,26,95,4,21,0,0,0,0,117,698
2,3,"Alemanha, República Democrática",0,0,0,0,4168,2630,12000,8250,...,3660,25467,6261,32605,2698,6741,7630,45367,4806,31853
3,4,Angola,0,0,0,0,0,0,0,0,...,345,1065,0,0,0,0,4068,4761,0,0
4,5,Anguilla,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,6,Antígua e Barbuda,0,0,0,0,0,0,0,0,...,219,1549,624,1864,805,2268,419,1866,383,1848
6,7,Antilhas Holandesas,280,207,4800,3705,3000,1936,0,0,...,0,0,0,0,0,0,0,0,0,0
7,8,Arábia Saudita,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,124,142
8,9,Argentina,0,0,0,0,0,0,0,0,...,0,0,1015,4176,6,13,480,3232,4545,36133
9,10,Aruba,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
# Transformação para formato longo
df_melted = pd.melt(df, id_vars=['Id', 'País'], var_name='Year_Metric', value_name='Value')

# Separação do ano e da métrica em duas novas colunas usando `str.extract`
df_melted[['Year', 'Metric']] = df_melted['Year_Metric'].str.extract(r'(\d{4})(\.1)?')
df_melted['Year'] = df_melted['Year'].astype(int)
df_melted['Metric'] = df_melted['Metric'].apply(lambda x: 'Value_USD' if x == '.1' else 'Quantity')

# Remoção da coluna Year_Metric
df_melted.drop(columns='Year_Metric', inplace=True)

# Pivot para formato largo e ajuste dos índices
df_pivot = df_melted.pivot(index=['Id', 'País', 'Year'], columns='Metric', values='Value').reset_index()

# Adição da coluna 'País de Origem'
df_pivot['País de Origem'] = 'Brazil'

# Renomeação das colunas
df_pivot.rename(columns={'País': 'País de Destino', 'Quantity': 'Quantidade em litros exportada', 'Value_USD': 'Valor em US$'}, inplace=True)

# Seleção final das colunas na ordem correta
df_pivot = df_pivot[['País de Origem', 'País de Destino', 'Year', 'Quantidade em litros exportada', 'Valor em US$']]

# Exibição das últimas 20 linhas
df_pivot.tail(20)


Metric,País de Origem,País de Destino,Year,Quantidade em litros exportada,Valor em US$
7378,Brazil,Vietnã,2004,0,0
7379,Brazil,Vietnã,2005,0,0
7380,Brazil,Vietnã,2006,2013,2362
7381,Brazil,Vietnã,2007,0,0
7382,Brazil,Vietnã,2008,0,0
7383,Brazil,Vietnã,2009,743,2143
7384,Brazil,Vietnã,2010,8820,9977
7385,Brazil,Vietnã,2011,0,0
7386,Brazil,Vietnã,2012,0,0
7387,Brazil,Vietnã,2013,0,0


In [19]:
df_filtered = df_pivot[df_pivot['Year'].between(2008, 2023)]
df_filtered.head()

Metric,País de Origem,País de Destino,Year,Quantidade em litros exportada,Valor em US$
38,Brazil,Afeganistão,2008,0,0
39,Brazil,Afeganistão,2009,0,0
40,Brazil,Afeganistão,2010,0,0
41,Brazil,Afeganistão,2011,0,0
42,Brazil,Afeganistão,2012,0,0


In [20]:
# Usar groupby com agregação diretamente em uma linha
df_grouped = df_filtered.groupby(['País de Origem', 'País de Destino'], as_index=False)[
    ['Quantidade em litros exportada', 'Valor em US$']
].sum()

# Renomear o resultado para garantir clareza, embora as colunas já estejam corretas
df_grouped.rename(columns={
    'Quantidade em litros exportada': 'Quantidade em litros exportada',
    'Valor em US$': 'Valor em US$'
}, inplace=True)

# Exibir as primeiras linhas do DataFrame final
df_grouped.head(10)

Metric,País de Origem,País de Destino,Quantidade em litros exportada,Valor em US$
0,Brazil,Afeganistão,11,46
1,Brazil,"Alemanha, República Democrática",913857,2578247
2,Brazil,Angola,168868,576189
3,Brazil,Anguilla,0,0
4,Brazil,Antilhas Holandesas,97002,142915
5,Brazil,Antígua e Barbuda,2487,9586
6,Brazil,Argentina,55557,257817
7,Brazil,Aruba,1350,2435
8,Brazil,Arábia Saudita,124,142
9,Brazil,Austrália,277139,482813


In [26]:
# Ordenar o DataFrame pela quantidade exportada em ordem decrescente
df_class = df_grouped.sort_values(by='Quantidade em litros exportada', ascending=False)

# Resetar o índice para manter a sequência após a ordenação
df_class = df_class.reset_index(drop=True)

# Formatar a coluna 'Quantidade em litros exportada' para adicionar vírgulas como separadores de milhares
df_class['Quantidade em litros exportada'] = df_class['Quantidade em litros exportada'].map(lambda x: f'{x:,}')

# Formatar a coluna 'Valor em US$' para adicionar vírgulas como separadores de milhares
df_class['Valor em US$'] = df_class['Valor em US$'].map(lambda x: f'{x:,}')



# Troca para alemanha e eslovaquia
df_class['País de Destino'] = df_class['País de Destino'].replace('Eslovaca, Republica', 'Eslováquia')
df_class['País de Destino'] = df_class['País de Destino'].replace('Alemanha, República Democrática', 'Alemanha')

# Exibir as primeiras linhas do DataFrame ordenado e formatado
df_class.head()

Metric,País de Origem,País de Destino,Quantidade em litros exportada,Valor em US$
0,Brazil,Rússia,"39,029,799","25,504,484"
1,Brazil,Paraguai,"32,995,148","44,236,294"
2,Brazil,Estados Unidos,"3,793,194","10,113,658"
3,Brazil,China,"2,583,375","4,929,621"
4,Brazil,Haiti,"2,351,248","3,198,869"


In [ ]:
# Convertentos os números formatados de volta em números inteiros para cálculos
df_class = df_class.sort_values(by='Quantidade em litros exportada', ascending=False).reset_index(drop=True)

# Formatando as colunas para incluir vírgulas 
df_class['Quantidade em litros exportada'] = df_class['Quantidade em litros exportada'].apply(lambda x: '{:,}'.format(x))
df_class['Valor em US$'] = df_class['Valor em US$'].apply(lambda x: '{:,}'.format(x))

# Convertendo de volta para números inteiros
df_class['Quantidade em litros exportada'] = df_class['Quantidade em litros exportada'].str.replace(',', '').astype(int)
df_class['Valor em US$'] = df_class['Valor em US$'].str.replace(',', '').astype(int)

# destinos de exportação p valor
df_top_value = df_class.nlargest(5, 'Valor em US$')
print("\nTop 5 Exportações por Valor (US$)")
display(df_top_value)

# destinos de exportação p quant
df_least_quantity = df_class[df_class['Quantidade em litros exportada'] > 0].nsmallest(5, 'Quantidade em litros exportada')
print("\nTop 5 Exportações por Quantidade")
display(df_least_quantity)

# valor/quantidade
df_class['Value-to-Quantity Ratio'] = df_class['Valor em US$'] / df_class['Quantidade em litros exportada']
df_class['Value-to-Quantity Ratio'] = df_class['Value-to-Quantity Ratio'].fillna(0)  # Prevenir divisão por zero
df_top_ratio = df_class.nlargest(5, 'Value-to-Quantity Ratio')
print("\nTop 5 Países com maior valor por litro exportado (US$)")
display(df_top_ratio)

# 4. Estatísticas descritivas
mean_quantity = df_class['Quantidade em litros exportada'].mean()
median_quantity = df_class['Quantidade em litros exportada'].median()
std_dev_quantity = df_class['Quantidade em litros exportada'].std()

mean_value = df_class['Valor em US$'].mean()
median_value = df_class['Valor em US$'].median()
std_dev_value = df_class['Valor em US$'].std()

print(f"\nMédia da Quantidade: {mean_quantity}")
print(f"Mediana da Quantidade: {median_quantity}")
print(f"Desvio Padrão da Quantidade: {std_dev_quantity}")

print(f"\nMédia do Valor: {mean_value}")
print(f"Mediana do Valor: {median_value}")
print(f"Desvio Padrão do Valor: {std_dev_value}")



Top 5 Exportações por Valor (US$)


Metric,País de Origem,País de Destino,Quantidade em litros exportada,Valor em US$,Value-to-Quantity Ratio
1,Brazil,Paraguai,32995148,44236294,1.340691
0,Brazil,Rússia,39029799,25504484,0.653462
2,Brazil,Estados Unidos,3793194,10113658,2.666264
3,Brazil,China,2583375,4929621,1.908210
6,Brazil,Reino Unido,1250877,4796011,3.834119



Top 5 Exportações por Quantidade


Metric,País de Origem,País de Destino,Quantidade em litros exportada,Valor em US$,Value-to-Quantity Ratio
125,Brazil,Tuvalu,2,4,2.000000
124,Brazil,Toquelau,3,10,3.333333
123,Brazil,Macau,7,6,0.857143
119,Brazil,Belice,9,29,3.222222
120,Brazil,Mauritânia,9,85,9.444444



Top 5 Países com maior valor por litro exportado (US$)


Metric,País de Origem,País de Destino,Quantidade em litros exportada,Valor em US$,Value-to-Quantity Ratio
87,Brazil,Eslováquia,585,16063,27.458120
97,Brazil,Bulgária,122,1610,13.196721
115,Brazil,Bangladesh,12,133,11.083333
102,Brazil,Croácia,55,607,11.036364
111,Brazil,Bermudas,16,153,9.562500



Média da Quantidade: 682637.3722627737
Mediana da Quantidade: 6329.0
Desvio Padrão da Quantidade: 4360903.385304156

Média do Valor: 887353.2262773722
Mediana do Valor: 13141.0
Desvio Padrão do Valor: 4465367.367785625
